## 문제 6

**Kaggle 형** train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8

...


**평가지표**

$RMSE(Y, \hat{Y}) = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i-\hat{y_i})^2}$

**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

# 모수적 모델 특히 선형모델에서는 가변수를 만들때 drop='first'가 꼭 필요합니다

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col='id')
df_test = pd.read_csv('test_prob.csv', index_col='id')
df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')

In [3]:
# 반복문을 구성하여 처리해 봅니다.

# 처리 내용을 정의합니다, (대상 변수명, 치환할 내용, 치환후 수준별 카운트)
repl_list = [
    ('cat3', {'B': 'C'}, [83634, 147361, 9005]), 
    ('cat4', {'A': 'B', 'D': 'B'}, [239397, 603]),
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [234203, 5145, 652]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'}, [4606, 19784, 214027, 1583]),
    ('cat8', {'B': 'G', 'F': 'E'}, [30338, 96743, 2953, 76085, 33881]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968])
]

for c, d, cnt in repl_list:
    print(c, d, cnt)
    s_repl = df_train[c].replace(d) # 치환을 합니다. (아직 반영은 하지 않습니다.)
    if not (s_repl.value_counts().sort_index() == cnt).all(): # 치환후 카운트를 체크합니다.
        print("Error", c, d, cnt, s_repl.value_counts().sort_index()) # 에러 내용을 출력합니다.
        break
    df_train[c] = s_repl # 치환한 결과를 반영합니다.
    # 일부러 cat9에 test에만 등장하는 수준을 만듭니다.
    if c != 'cat9':
        df_test[c] = df_test[c].replace(d) # 테스트에 대해서도 반영합니다.

cat3 {'B': 'C'} [83634, 147361, 9005]
cat4 {'A': 'B', 'D': 'B'} [239397, 603]
cat6 {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'} [234203, 5145, 652]
cat7 {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'} [4606, 19784, 214027, 1583]
cat8 {'B': 'G', 'F': 'E'} [30338, 96743, 2953, 76085, 33881]
cat9 {'C': 'H', 'D': 'B', 'E': 'L'} [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968]


In [4]:
# 문제 4번을 활용하기 위해 반듭니다.
df_train['targetA'] = df_train['target'] <= 7.45

In [5]:
q = [i for i in np.arange(0, 1.01, 0.01)]
# 나머지 변수에 대해서도 해당 파생 변수를 만들어 줍니다.
for i in range(0, 14):
    col = 'cont{}'.format(i)
    qt = df_train[col].quantile(q)
    qt.iloc[[0, -1]] = [-np.inf, np.inf]
    q_cut = pd.cut(df_train[col], bins=qt)
    q_mean = df_train.groupby(q_cut)['target'].mean()
    df_train[col + '_q'] = q_cut.map(q_mean).astype('float')
    df_test[col + '_q'] = pd.cut(df_test[col], bins=qt).map(q_mean).astype('float')

In [6]:
from scipy.stats import norm

mu_A, s_A = 6.769, 0.616
mu_B, s_B = 8.123, 0.527

df_train_clf = df_train.assign(
    # 귀무가설 : target은 A입니다, 대립가설: target은 B입니다.
    prob_A = 1 - norm.cdf(df_train['target'], loc=mu_A, scale=s_A),
    # 귀무가설 : target은 B입니다, 대립가설: target은 A입니다.
    prob_B = norm.cdf(df_train['target'], loc=mu_B, scale=s_B)
)
df_train_clf = df_train_clf.query('prob_B < 0.01 or prob_A < 0.01').copy()

In [7]:
# 공통
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import ShuffleSplit, KFold 

cv = KFold(n_splits=5, random_state=123)
ss = ShuffleSplit(n_splits=1, train_size=0.8, random_state=123)
neg_rmse = make_scorer(lambda y, y_hat: -mean_squared_error(y, y_hat) ** 0.5)
X = df_test.columns.tolist() + ['cont{}_q'.format(i) for i in range(14)] + ['targetA_prob']

# 위에서 발생한 leak을 바로 잡아 교차검증을 합니다.
q = [i for i in np.arange(0, 1.01, 0.01)]
def eval_model(model, sp):
    score_train, score = list(), list()
    for train_idx, test_idx in sp.split(df_train):
        df_cv_train, df_cv_test = df_train.iloc[train_idx].copy(), df_train.iloc[test_idx].copy()
        # 검증셋에서 train으로 파생변수를 만들고
        # 검증셋의 test(겹외셋)에 검증셋의 train으로 만든 통계값(mean)을 반영합니다.
        for i in range(0, 14):
            col = 'cont{}'.format(i)
            qt = df_cv_train[col].quantile(q)
            qt.iloc[[0, -1]] = [-np.inf, np.inf]
            q_cut = pd.cut(df_cv_train[col], bins=qt)
            q_mean = df_cv_train.groupby(q_cut)['target'].mean()
            df_cv_train[col + '_q'] = q_cut.map(q_mean).astype('float')
            df_cv_test[col + '_q'] = pd.cut(df_cv_test[col], bins=qt).map(q_mean).astype('float')
        model.fit(df_cv_train[X], df_cv_train['target'])
        score_train.append(-(mean_squared_error(df_cv_train['target'], model.predict(df_cv_train[X]))) ** 0.5)
        score.append(-(mean_squared_error(df_cv_test['target'], model.predict(df_cv_test[X]))) ** 0.5)
    return score_train, score

def choose_model(model):
    model.fit(df_train[X], df_train['target'])
    prd = model.predict(df_test[X])
    pd.DataFrame({
        'target': prd
    }, index=df_test.index).to_csv('answer6.csv')
    return prd 

In [9]:
# 문제 3번 모델을 만듭니다. targetA일 확률을 활용할 예정입니다.
import xgboost as xgb

# Tree 계열(Tree 계열 Ensemble 모델 포함)은 무시(handle_unknown='ignore')해도 됩니다.
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}'.format(i) for i in range(14)])
])
X_xgb = ['cont{}'.format(i) for i in range(14)] + ['cat{}'.format(i) for i in range(10)]
clf_xgb = make_pipeline(
    ct,
    xgb.XGBClassifier(
        max_depth = 2, # 트리의 최대 깊이 2
        reg_alpha = 0.1, # L1 규제 0.1
        reg_lambda = 0.1, # L2 규제 0.1
        colsample_bytree=0.25, # 트리 당 컬럼 샘플링 비율 0.25
        n_estimators=500, # 트리의 수 500
        random_state=123, # random_state 123
    )
)

X_xgb = ['cat{}'.format(i) for i in range(10)] + ['cont{}'.format(i) for i in range(14)]
clf_xgb.fit(df_train_clf[X_xgb], df_train_clf['targetA'])
df_train['targetA_prob'] = clf_xgb.predict_proba(df_train[X_xgb])[:, 1]
df_test['targetA_prob'] = clf_xgb.predict_proba(df_test[X_xgb])[:, 1]

In [11]:
# Baseline 모델을 만들어 봅니다.
# handle_unknown을 ignore로 하여 등장하지 않은 수준에 대해 에러가 발생하지 않도록 합니다.
# 이 경우 해당 변수의 가변수 모든 값은 0이 됩니다.
from sklearn.linear_model import LinearRegression
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}'.format(i) for i in range(14)])
])
X_lr = ['cat{}'.format(i) for i in range(10)] + ['cont{}'.format(i) for i in range(14)]
reg_lr = make_pipeline(ct, LinearRegression())
scores_train, scores_test = eval_model(reg_lr, cv)
scores_train, scores_test, np.mean(scores_test)

([-0.8640923487492325,
  -0.8622410903241682,
  -0.8636089940936378,
  -0.8628898930428618,
  -0.8622645403885227],
 [-0.8589375958480246,
  -0.8664481935452925,
  -0.8608939498501563,
  -0.8637802543667048,
  -0.8662598490782522],
 -0.863263968537686)

In [12]:
prd = choose_model(reg_lr)

In [13]:
# 검증시에는 문제가 보이지 않지만
# 제출한 test 결과에는 심한 결합이 보입니다.
# 변수마다 하나의 가변수를 제외하지 않아, 완전한 다중공선성에 의한
# 모델의 불안정성이 심하게되어 문제를 일으킨 것입니다.
mean_squared_error(df_ans['target'], prd) ** 0.5

2035002760.747577

In [14]:
# 회귀 계수에 큰 값이 채워져 있습니다.
# 불안정한 모델이 되었습니다.
reg_lr.fit(df_train[X], df_train['target'])
reg_lr[1].coef_

array([ 1.02275370e+11,  1.02275370e+11,  6.97260841e+09,  6.97260841e+09,
       -6.47470121e+09, -6.47470121e+09, -1.16364952e+10, -1.16364952e+10,
       -1.16364952e+10,  5.92733932e+10,  5.92733932e+10, -6.05352419e+09,
       -6.05352419e+09, -6.05352419e+09, -6.05352419e+09, -6.13632917e+09,
       -6.13632917e+09, -6.13632917e+09, -2.13267137e+10, -2.13267137e+10,
       -2.13267137e+10, -2.13267137e+10,  1.26928173e+10,  1.26928173e+10,
        1.26928173e+10,  1.26928173e+10,  1.26928173e+10,  5.60824635e+10,
        5.60824635e+10,  5.60824635e+10,  5.60824635e+10,  5.60824635e+10,
        5.60824635e+10,  5.60824635e+10,  5.60824635e+10,  5.60824635e+10,
        5.60824635e+10,  5.60824635e+10,  5.60824635e+10, -2.84263611e-01,
        1.46797180e-01,  1.07498169e-02, -2.32238770e-02, -2.70361900e-02,
       -2.35549927e-01,  1.22724533e-01,  7.58972168e-02,  3.08395386e-01,
        1.33668900e-01, -1.05171204e-01,  2.24174500e-01,  7.12165833e-02,
       -2.58636475e-02])

In [15]:
# 따라서 Linear과 같은 모수적 모델에서는 모델의 안정성을 위해, 
# drop='first'를 반드시 해주어야 합니다. 
# Test만 등장하는 수준을 제거해 줍니다.

# Test에만 등장하는 수준을 봅니다.
pd.concat([
    df_train[[k[0] for k in repl_list]].apply(lambda x: set(x.unique())).rename('train_level'),
    df_test[[k[0] for k in repl_list]].apply(lambda x: set(x.unique())).rename('test_level')
], axis=1).apply(lambda x: x['test_level'] - x['train_level'], axis=1)

cat3           {}
cat4           {}
cat6           {}
cat7           {}
cat8           {}
cat9    {C, D, E}
dtype: object

In [16]:
# 미등장 수준을 가장 자주 등장하는 수준으로 바꿉니다.
df_test['cat9'] = df_test['cat9'].replace({'D': np.nan, 'E': np.nan, 'C': np.nan})
df_test['cat9'] = df_test['cat9'].fillna(df_train['cat9'].mode().iloc[0])

In [17]:
# Baseline 모델을 만들어 봅니다.
# drop='first' 설정을 해줍니다.
from sklearn.linear_model import LinearRegression
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}'.format(i) for i in range(14)])
])
X_lr = ['cat{}'.format(i) for i in range(10)] + ['cont{}'.format(i) for i in range(14)]
reg_lr = make_pipeline(ct, LinearRegression())
scores_train, scores_test = eval_model(reg_lr, cv)
scores_train, scores_test, np.mean(scores_test)

([-0.8640766554697877,
  -0.8622215542863032,
  -0.8635901024339211,
  -0.8628727191950373,
  -0.8622518628759744],
 [-0.8589626898717829,
  -0.8664070309294921,
  -0.8608483019953761,
  -0.8637481914465037,
  -0.8662619974502677],
 -0.8632456423386845)

In [18]:
# 문제가 해결됩니다.
prd = choose_model(reg_lr)
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8657268588416211

In [19]:
# 회귀 계수에 큰 값이 채워져 있습니다.
# 문제가 해소되었습니다.
reg_lr.fit(df_train[X], df_train['target'])
reg_lr[1].coef_

array([-0.14737833,  0.16983335,  0.24147158,  0.10693804,  0.03994362,
        0.15713566,  0.03681214, -0.17486971,  0.03074087,  0.21519861,
        0.36870383, -0.00597853, -0.05547379, -0.05870304, -0.00092302,
        0.05854559,  0.07181603, -0.10018803,  0.03508515,  0.13365193,
        0.1647826 ,  0.08022279, -0.00404722,  0.02865399,  0.09003857,
        0.21092791,  0.09276305,  0.09655687,  0.07017521, -0.28413913,
        0.14685393,  0.01069457, -0.02323074, -0.02707619, -0.23571329,
        0.12262252,  0.07588697,  0.30830852,  0.13373422, -0.10509757,
        0.22408617,  0.0712212 , -0.02577207])